### building a class for Autoregressions  
The code below builds a basic class for fitting,forecasting and simulating autoregressive processes.

In [2]:
import numpy as np
import numpy.random as rnd
from numpy import hstack,nan,tile,vstack,zeros
import matplotlib.pyplot as plt

try:
    import seaborn
except ImportError:
    pass

c:\python27\lib\site-packages\matplotlib\__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [8]:
class AR(object):
    '''
    The __init__ takes 2 optional arguments,and initializes a large number of 
    private variables,which will either act as backing for properties or hold 
    intermediate values.
    '''
    def __init__(self,data=None,ar_order=1):
        self._data=data
        self._ar_order=ar_order
        self._sigma2=1
        self._T=None
        self._X=None
        self._y=None
        self._roots=None
        self._abs_roots=None
        self._fit_values=None
        self._parameters=None
        self._errors=None
        if data is not None:
            self._generate_regressor_array()
    
    '''
    Properties
    The class has a many properties,and most simply return the value of a private
    variable with the same name.
    '''
    @property
    def data(self):
        return self._data
    
    @property
    def ar_order(self):
        return self._ar_order
    
    @property
    def parameters(self):
        return self._parameters
    
    @property
    def sigma2(self):
        return self._sigma2
    
    @property
    def roots(self):
        return self._roots
    
    @property
    def abs_roots(self):
        return self._abs_roots
    
    @property
    def fit_values(self):
        if self._fit_values is None:
            self.estimate()
        return self._fit_values
    
    '''
    Most of the properties are read-only.data,parameters,and sigma2 
    can be directly set.Whenever data is set,the regressors are updated,
    and,when the object has parameters,other quantities,such as the fitted
    values,are computed.
    '''
    @data.setter
    def data(self,value):
        self._data=value
        self._generate_regressor_array()
        if self._parameters is not None:
            self._update_values()
    
    @parameters.setter
    def paramters(self,value):
        if value.ndim not in(1,2):
            raise ValueError('parameters must be a vector')
        value=value.ravel()[:,None]
        if value.shape[0]!=(self._ar_order+1):
            raise ValueError('parameters must have {0:d} elements' \
                            .format(self._ar_order+1))
        self._parameters=value
        self._update_roots()
    
    @sigma2.setter
    def sigma2(self,value):
        if value<=0.0:
            raise ValueError('sigma2 must be positive')
        self._sigma2=value
    
    '''
    Private methods
    There are three private methods which:
        1.Generate the regressor and regressand arrays.This is called
        whenever data is changed.
        
        2.Update the roots and absolute roots.This is called whenever
        parameters are set.
        
        3.Compute fitted values,errors and estimate the variance.This
        is called whenever the model is estimated or the data is changed
    '''
    def _generate_regressor_array(self):
        p=self._ar_order
        T=self._T=len(self._data)
        x=np.ones((T-p,p+1))
        y=self._data[:,None]
        for i in range(p):
            x[:,[i+1]]=y[p-i-1:T-i-1,:]
        self._y=self.data[p:,None]
        self._X=x
        
    def _update_roots(self):
        if self._ar_order>0:
            char_equation=np.concatenate(([1],-1.0*self._paramters[1:].ravel()))
            self._roots=np.roots(char_equation)
            self._abs_roots=np.absolute(self._roots)
        else:
            self._roots = None
            self._abs_roots = None
        def _update_values(self):
            fv = self._X.dot(self._parameters)
            e = self._y - fv
            self._sigma2 = np.dot(e.T, e) / len(e)
            self._errors = vstack((tile(nan, (self._ar_order, 1)), e))
            self._fit_values = vstack((tile(nan, (self._ar_order, 1)), fv))

    '''
    Methods
    There are 5 methods:
    estimate
    forcast
    forecast_plot
    hedge_plot
    simulate-Simulate a time series with the same parameters are the 
             model using Gaussian shocks
    '''    
    def estimate(self, insample=None):
        x = self._X
        y = self._y
        y = y[:, None]
        p = self._ar_order
        if insample is not None:
            x = x[:(insample - p),:]
            y = y[:(insample - p),:]
        xpxi = np.linalg.pinv(x)
        self.parameters = xpxi.dot(y)

    
    def forecast(self, h=1, insample=None):
        tau = self._X[:, 1:].shape[0]
        forecasts = hstack((self._X[:, :0:-1], zeros((tau, h))))
        p = self._ar_order
        params = self._parameters
        for i in range(h):
            forecasts[:, p + i] = params[0]
            for j in range(p):
                forecasts[:, p + i] += params[j + 1] * forecasts[:, p + i - (j + 1)]
        
        forecasts = vstack((tile(nan, (p, p + h)), forecasts))
        if insample is not None:
            forecasts[:insample, :] = nan
        
        return forecasts[:, p:]
            
            
    def forecast_plot(self, h=1, show=True, show_errors=False):
        forecasts = self.forecast(h=h)
        T = self._T
        p = self._ar_order
        aligned = zeros((T + h, 2))
        aligned[:T, 0] = self._data

        aligned[-T:, 1] = forecasts[:, -1]
        aligned = aligned[p:T, :]
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        if show_errors:
            ax.plot(aligned[:, 0] - aligned[:, 1])
        else:
            ax.plot(aligned)

        if show:
            plt.show(fig)

        return fig
            
    
    def hedgehog_plot(self, h=1, show=True, skip=0):
        forecasts = self.forecast(h=h)
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(self._data)
        ax.hold(True)
        data = self._data
        for i in range(0, self._T, skip + 1):
            x = i + np.arange(h + 1)
            y = hstack((data[[i]], forecasts[i]))
            ax.plot(x, y, 'r')

        ax.hold(False)
        ax.autoscale(tight='x')
        fig.tight_layout()
        if show:
            plt.show(fig)

        return fig
            
            
    def simulate(self, T=500, burnin=500):
        tau = T + burnin + self._ar_order
        e = rnd.standard_normal((tau,))
        y = zeros((tau,))
        p = self._ar_order
        for t in range(p, tau):
            y[t] = self._parameters[0]
            for i in range(p):
                y[t] += self._parameters[i + 1] * y[t - i - 1]
            y[t] += e[t]
            
        return y[-T:]            